In [7]:
def adjust_state(state, rotation=0, flip_horizontal=False, flip_vertical=False):
    direction_index = {'UP': 0, 'RIGHT': 1, 'DOWN': 2, 'LEFT': 3}
    directions = ['UP', 'RIGHT', 'DOWN', 'LEFT']  # Define the initial order of directions before rotation

    # Normalize and apply clockwise rotation
    rotation = (rotation // 90) % 4  # Normalize the rotation to one of 0, 0, 2, or 3
    directions = directions[-rotation:] + directions[:-rotation]  # Rotate the list using slicing

    # Handle horizontal flip
    if flip_horizontal:
        directions[1], directions[3] = directions[3], directions[1]  # Swap 'RIGHT' and 'LEFT'

    # Handle vertical flip
    if flip_vertical:
        directions[0], directions[2] = directions[2], directions[0]  # Swap 'UP' and 'DOWN'

    # Update the state according to the new direction list
    new_state = [0] * 6
    for i, dir in enumerate(directions):
        new_state[i] = state[direction_index[dir]]
    new_state[4] = state[4]  # 'CENTER' remains unchanged
    new_state[5] = state[5]  # 'BOMBS_LEFT' remains unchanged

    return new_state

In [8]:
state = ['U', 'R', 'D', 'L', 'C', 'B']

adjust_state(state, 0, True)

['L', 'U', 'R', 'D', 'C', 'B']

In [9]:
import numpy as np

def find_crates_neighbors(field):
    # 通过切片获取中心区域及其相邻的四个部分
    center = field[1:-1, 1:-1]
    up = field[:-2, 1:-1]
    down = field[2:, 1:-1]
    left = field[1:-1, :-2]
    right = field[1:-1, 2:]

    # 生成一个布尔数组，其中True表示中心位置为0且至少有一个邻居为1
    result = (center == 0) & ((up == 1) | (down == 1) | (left == 1) | (right == 1))
    
    # 获取结果的索引位置，并调整索引以匹配原始数组
    positions = np.argwhere(result)
    return positions + 1  # 加1以调整到原始数组的索引

# 示例数组，最外圈设为-1
field = np.array([[-1, -1, -1, -1, -1],
                  [-1, 0, 0, 0, -1],
                  [-1, 0, 0, 1, -1],
                  [-1, 0, 1, 0, -1],
                  [-1, -1, -1, -1, -1]])

# 调用函数
positions = find_crates_neighbors(field)
print("符合条件的位置：\n", positions)


符合条件的位置：
 [[1 1]
 [1 3]
 [2 2]
 [3 1]
 [3 3]]


In [11]:
import os
import json
import random
from collections import deque
import settings as s

import numpy as np


def find_safe_positions(game_state, danger, max_steps=5):
    field, explosion_map, bombs, _, others, pos = (
        game_state['field'], game_state['explosion_map'], game_state['bombs'],
        game_state['coins'], game_state['others'], game_state['self'][-1]
    )
    directions = [(0, -1), (1, 0), (0, 1), (-1, 0), (0, 0)]
    first_step_to_safes = {}
    q = deque()
    q.append((pos[0], pos[1], 0, None))
    while q:
        x, y, steps, first_step = q.popleft()
        if (danger[x][y] > 0 and s.BOMB_TIMER - danger[x][y] < steps <= s.BOMB_TIMER - danger[x][y] + s.EXPLOSION_TIMER) \
            or explosion_map[x][y] > steps: # explosion while agent passing by
            continue
        if steps == max_steps:
            if first_step not in first_step_to_safes:
                first_step_to_safes[first_step] = []
            if (x, y) not in first_step_to_safes[first_step]:
                first_step_to_safes[first_step].append((x, y))
            continue
        for i, (dx, dy) in enumerate(directions):
            nx, ny = x + dx, y + dy
            if 0 <= nx < field.shape[0] and 0 <= ny < field.shape[1]:
                if field[nx, ny] != 0:
                    continue
                if any((nx, ny) == other[-1] for other in others) and steps == 0:
                    continue
                bomb_prevent = any((nx, ny) == (bx, by) and timer >= steps for (bx, by), timer in bombs)
                if bomb_prevent:
                    continue
                new_first_step = i if first_step is None else first_step
                if steps < max_steps:
                    q.append((nx, ny, steps + 1, new_first_step))
    return first_step_to_safes

In [12]:
def calculate_steps(game_state, pos, objects, danger):
    field, explosion_map, bombs, _, others, (x, y) = (
        game_state['field'], game_state['explosion_map'], game_state['bombs'],
        game_state['coins'], game_state['others'], game_state['self'][-1]
    )
    distances = np.full((len(objects),), np.inf)
    target_index_map = {tuple(obj): idx for idx, obj in enumerate(objects)}
    directions = [(0, -1), (1, 0), (0, 1), (-1, 0), (0, 0)] 

    q = deque()
    q.append((pos[0], pos[1], 0))

    visited = set()
    visited.add(tuple(pos))
    
    # BFS
    while q:
        x, y, steps = q.popleft()
        if (danger[x][y] > 0 and s.BOMB_TIMER - danger[x][y] < steps <= s.BOMB_TIMER - danger[x][y] + s.EXPLOSION_TIMER) \
            or explosion_map[x][y] > steps: # explosion while agent passing by
            continue
        if (x, y) in target_index_map:
            idx = target_index_map[(x, y)]
            distances[idx] = steps
            del target_index_map[(x, y)]
            if not target_index_map:
                break
        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            if 0 <= nx < field.shape[0] and 0 <= ny < field.shape[1] and (nx, ny) not in visited:
                if field[nx, ny] != 0:
                    continue
                if any((nx, ny) == other[-1] for other in others) and steps == 0:
                    continue
                bomb_prevent = any((nx, ny) == (bx, by) and timer >= steps for (bx, by), timer in bombs)
                if bomb_prevent:
                    continue
                visited.add((nx, ny))
                q.append((nx, ny, steps + 1))
    
    return distances

In [18]:
def find_crates_neighbors(field):
    center = field[1:-1, 1:-1]
    up = field[:-2, 1:-1]
    down = field[2:, 1:-1]
    left = field[1:-1, :-2]
    right = field[1:-1, 2:]
    result = (center == 0) & ((up == 1) | (down == 1) | (left == 1) | (right == 1))
    positions = np.argwhere(result)
    return positions + 1  # back to ori index

def look_for_target(game_state, features, safe_positions, danger):
    field, _, _, coins, others, (x, y) = (
        game_state['field'], game_state['explosion_map'], game_state['bombs'],
        game_state['coins'], game_state['others'], game_state['self'][-1]
    )
    candidates = np.array(list(safe_positions.keys()))
    directions = [(x, y - 1), (x + 1, y), (x, y + 1), (x - 1, y), (x, y)]
    weights = {"crates": 1, "coins": 50, "enemy": 1}

    if not candidates.size:
        return

    # Precompute positions for crates, coins, and enemies
    object_positions = {
        "crates": find_crates_neighbors(field),
        "coins": np.array(coins),
        "enemy": np.array([op[-1] for op in others])
    }

    scores = np.zeros(len(candidates))
    # Precompute distances from current positions to all types of objects
    for object_name, positions in object_positions.items():
        if positions.size:
            for i, index in enumerate(candidates):
                direction = np.array(directions[index])
                distances = calculate_steps(game_state, direction, positions, danger)
                if distances.size:
                    scores[i] += np.sum(weights[object_name] / (distances + 1))
                    if danger[x][y] > 0:
                        scores[i] += len(safe_positions[index]) * 20

    # Determine the best candidates with the maximum score
    best_indices = candidates[scores == scores.max()]
    print(scores)

    if best_indices.size:
        chosen_index = np.random.choice(best_indices)
        features[chosen_index] = "target"

In [19]:
game_state = {}
game_state['field'] = np.array(
[[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
 [-1, 0,  0, 0,  0, 0,  0, 0,  0, 0,  0, 0,  0, 0,  0, 0, -1],
 [-1, 0, -1, 0, -1, 0, -1, 0, -1, 0, -1, 0, -1, 1, -1, 0, -1],
 [-1, 0,  0, 0,  0, 0,  1, 0,  1, 0,  1, 0,  0, 0,  0, 0, -1],
 [-1, 0, -1, 0, -1, 0, -1, 0, -1, 0, -1, 0, -1, 1, -1, 0, -1],
 [-1, 0,  0, 0,  0, 0,  0, 0,  0, 0,  0, 0,  0, 0,  0, 0, -1],
 [-1, 1, -1, 0, -1, 0, -1, 0, -1, 0, -1, 0, -1, 1, -1, 0, -1],
 [-1, 0,  0, 0,  0, 0,  0, 0,  0, 0,  0, 0,  0, 0,  0, 0, -1],
 [-1, 1, -1, 0, -1, 0, -1, 0, -1, 0, -1, 0, -1, 1, -1, 0, -1],
 [-1, 0,  0, 0,  0, 0,  0, 0,  0, 0,  0, 0,  0, 0,  0, 0, -1],
 [-1, 0, -1, 0, -1, 0, -1, 0, -1, 0, -1, 0, -1, 0, -1, 0, -1],
 [-1, 0,  0, 0,  0, 0,  0, 0,  0, 0,  0, 0,  0, 0,  0, 0, -1],
 [-1, 0, -1, 0, -1, 0, -1, 0, -1, 0, -1, 0, -1, 0, -1, 0, -1],
 [-1, 0,  0, 0,  0, 0,  0, 0,  1, 0,  1, 0,  1, 0,  1, 0, -1],
 [-1, 0, -1, 0, -1, 1, -1, 0, -1, 0, -1, 0, -1, 0, -1, 0, -1],
 [-1, 0,  0, 0,  0, 0,  0, 0,  0, 0,  0, 0,  0, 0,  0, 0, -1],
 [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]]
)
game_state['explosion_map'] = np.zeros((17, 17))
game_state['bombs'] = []
game_state['coins'] = []
game_state['others'] = []
game_state['self'] = [(9, 11)]
dangers = np.zeros((17, 17))
features = ['free', 'free', 'free', 'free', 'free']

safe_positions = find_safe_positions(game_state, dangers)
look_for_target(game_state, features, safe_positions, dangers)

[2.2797619  2.33869048 2.41011905 2.37619048 2.41027861]


In [17]:
features

['free', 'free', 'free', 'free', 'target']